In [1]:
import numpy as np
import nltk
import json
import random
nltk.download('punkt')
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
def tokenize(sentence):
    """
    split sentence into array of words/tokens
    a token can be a word or punctuation character, or number
    """
    return nltk.word_tokenize(sentence)


def stem(word):
    """
    stemming = find the root form of the word
    examples:
    words = ["organize", "organizes", "organizing"]
    words = [stem(w) for w in words]
    -> ["organ", "organ", "organ"]
    """
    return stemmer.stem(word.lower())


def bag_of_words(tokenized_sentence, words):
    """
    return bag of words array:
    1 for each known word that exists in the sentence, 0 otherwise
    example:
    sentence = ["hello", "how", "are", "you"]
    words = ["hi", "hello", "I", "you", "bye", "thank", "cool"]
    bog   = [  0 ,    1 ,    0 ,   1 ,    0 ,    0 ,      0]
    """
    # stem each word
    sentence_words = [stem(word) for word in tokenized_sentence]
    # initialize bag with 0 for each word
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words: 
            bag[idx] = 1

    return bag

In [3]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.l2 = nn.Linear(hidden_size, hidden_size) 
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        # no activation and no softmax at the end
        return out

In [4]:
file_path = '/content/intents.json'
with open(file_path, 'r') as f:
    intents = json.load(f)

In [5]:
all_words = []
tags = []
xy = []
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    tag = intent['tag']
    # add to tag list
    tags.append(tag)
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = tokenize(pattern)
        # add to our words list
        all_words.extend(w)
        # add to xy pair
        xy.append((w, tag))

In [6]:
# stem and lower each word
ignore_words = ['?', '.', '!']
all_words = [stem(w) for w in all_words if w not in ignore_words]
# remove duplicates and sort
all_words = sorted(set(all_words))
tags = sorted(set(tags))

print(len(xy), "patterns")
print(len(tags), "tags:", tags)
print(len(all_words), "unique stemmed words:", all_words)

236 patterns
46 tags: ['Company_details', 'achievements', 'cancelations-refunds', 'change', 'class_enroll', 'classes_levels', 'coding', 'coding_language', 'contact', 'course_enroll', 'course_levels', 'coursecontents', 'courseprice', 'courses', 'difference', 'eligible', 'goodbye', 'greeting', 'help', 'liveclassescontents', 'liveclassesprice', 'mentors', 'missed_class', 'monitor', 'mulitple_courses', 'noanswer', 'offers', 'options', 'outcome', 'partners', 'payment_platforms', 'prerequisite', 'programming_language', 'size', 'special', 'specific_questions', 'specific_questions1', 'specific_questions2', 'specific_questions3', 'specific_questions4', 'specific_questions5', 'statistics', 'suggestions', 'thanks', 'time', 'values']
365 unique stemmed words: ["'s", ',', '..', '...', '.can', '1', '1-1', '10', '11', '12', '13', '14', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'about', 'accommod', 'achiev', 'across', 'ad', 'advanc', 'after', 'age', 'aid', 'all', 'alloc', 'am', 'an', 'ani', 'anyon'

In [7]:
# create training data
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    # X: bag of words for each pattern_sentence
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    # y: PyTorch CrossEntropyLoss needs only class labels, not one-hot
    label = tags.index(tag)
    y_train.append(label)

In [8]:
X_train = np.array(X_train)
y_train = np.array(y_train)

In [9]:
# Hyper-parameters 
num_epochs = 1000
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)
print(input_size, output_size)

365 46


In [10]:
class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = NeuralNet(input_size, hidden_size, output_size).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)
        
        # Forward pass
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (epoch+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [100/1000], Loss: 0.0567
Epoch [200/1000], Loss: 0.0155
Epoch [300/1000], Loss: 0.0010
Epoch [400/1000], Loss: 0.2626
Epoch [500/1000], Loss: 0.0001
Epoch [600/1000], Loss: 0.0000
Epoch [700/1000], Loss: 0.2096
Epoch [800/1000], Loss: 0.0000
Epoch [900/1000], Loss: 0.1843
Epoch [1000/1000], Loss: 0.0000


In [12]:
bot_name = "Sam"
print("Let's chat! (type 'quit' to exit)")
while True:
  # sentence = "do you use credit cards?"
  sentence = input("You: ")
  if sentence == "quit":
      break

  sentence = tokenize(sentence)
  X = bag_of_words(sentence, all_words)
  X = X.reshape(1, X.shape[0])
  X = torch.from_numpy(X).to(device)

  output = model(X)
  _, predicted = torch.max(output, dim=1)

  tag = tags[predicted.item()]

  probs = torch.softmax(output, dim=1)
  prob = probs[0][predicted.item()]
  if prob.item() > 0.75:
      for intent in intents['intents']:
          if tag == intent["tag"]:
              print(f"{bot_name}: {random.choice(intent['responses'])}")
  else:
      print(f"{bot_name}: I do not understand...")

Let's chat! (type 'quit' to exit)
You: hi
Sam: Hello, thanks for asking
You: how could you help me?
Sam: Offering support for little coders, Live Interactive classes
You: which languages do you teach?
Sam: It completely depends on the age and interest of an individual.Students need to be guided properly.You can contact us regarding this.
You: how should i enroll?
Sam: Choose Course which you want to enroll yourself.Fill the details Make your payment and enjoy CODING
You: thanks
Sam: My pleasure
You: bye!
Sam: Have a nice day
You: quit
